# Use litgen online

This page shows an example of conversion of C++ code into C++ bindings code and python stubs.
You can open it in an online interactive environment ([mybinder](https://mybinder.org)), where you can edit the code below.

1. To launch the online tool, click on the rocket icon at the top right of this page, or use [this link](https://mybinder.org/v2/gh/pthom/litgen/main?urlpath=lab/tree/litgen-book/01_05_05_online.ipynb)
   
2. The interactive environment will open in a new tab. Wait for it to be ready
   
3. Once it is launched, click on "Run Cell" to run each of the cells: 
   
  ![Run cell](images/run_cell.jpg)

You can edit the C++ code as well as the options below. The generated code will be displayed under the cell, once you clicked "Run Cell".
You will then be able to see:
- The generated python stubs (aka declarations)
- The generated C++ bindings code for pybind11
- The generated C++ bindings code for nanobind
- The generated glue code (if needed)

In [2]:
import litgen
from litgen.demo import litgen_demo

options = litgen.LitgenOptions()

# This namespace will not be outputted as a python module
options.namespaces_root = ["MyNamespace"]
# All functions, modules and namespaces names are converted to snake case
options.python_convert_to_snake_case = True
# This is an API marker in the C++ code (for shared libraries code)
options.srcmlcpp_options.functions_api_prefixes = "^MY_API$"
# Also create bindings for functions that do not have the API marker
options.fn_exclude_non_api = False
# Optional comment that can be added to non API functions
options.fn_non_api_comment = ""
# "Box" immutable functions parameter when they should be modifiable
options.fn_params_replace_modifiable_immutable_by_boxed__regex = r".*"
# Which numeric and string preprocessor do we want to export
options.macro_define_include_by_name__regex = "^MY_"

code = """
// This namespace is not outputed as a submodule, since it is marked as Root (see options.namespaces_root)
namespace MyNamespace 
{
    // Multiplies a list of double by a given factor, returns updated list
    std::vector<double> MultiplyDoubles(const std::vector<double>& v, float k);

    // changes the value of the bool parameter (passed by modifiable reference)
    // (This function will use a BoxedBool in the python code, so that its value can be modified)
    void SwitchBoolValue(bool &v);
    
    // Standalone comment blocs are also exported

    // This function includes an API marker which will be ignored when generating the bindings
    MY_API int MySubstract(int a, int b); // eol doc is also included in bindings doc

    namespace MyMath // This namespace is not ignored and introduces a new python module
    {
        // div and mul: divide or multiply float numbers
        // (This comment concerns the two grouped 
        // functions below, and will be exported as such)
        float Div(float a, float b); // Divide
        float Mul(float a, float b); // Multiply
    }

    // Stores Pixel coordinates
    struct Pixel
    {
        // Coordinates
        double x = 2., y = 3.;

        // Draw a pixel
        void Draw(Image& i);

        private:
        double _Norm(); // this will not be exported as it is private
    };

    // This macro value be exported, as it matches the regex macro_define_include_by_name__regex
    #define MY_VALUE 123
}
"""

litgen_demo.demo(
    options, code, show_cpp=False, show_pydef=True, height=80
)